In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Crawler

Mounted at /content/gdrive
/content/gdrive/MyDrive/Crawler


In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 6.3 MB 4.6 MB/s 
     |████████████████████████████████| 5.3 MB 37.8 MB/s 
     |████████████████████████████████| 27.4 MB 45.8 MB/s 
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
     |████████████████████████████████| 1.5 MB 38.8 MB/s 
     |████████████████████████████████| 10.4 MB 20.9 MB/s 
     |████████████████████████████████| 14.5 MB 42.4 MB/s 
     |████████████████████████████████| 787 kB 47.9 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 374 kB 48.2 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=cbd84e42567814bfa5bfc3a2a99d9cbbaca26322d65a5b73144dfc30efa035c3
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalle

## Convert PDF to CSV

In [ ]:
!python -m processor.run -m pdf2csv -i pos_reports -o pos_csv

(0) processing : pos_reports/16357.pdf
... Saving output csv : pos_csv/16357.csv
(1) processing : pos_reports/4047.pdf
... Saving output csv : pos_csv/4047.csv
(2) processing : pos_reports/86.pdf
... Saving output csv : pos_csv/86.csv
(3) processing : pos_reports/16456.pdf
... Saving output csv : pos_csv/16456.csv
(4) processing : pos_reports/16468.pdf
... Saving output csv : pos_csv/16468.csv
(5) processing : pos_reports/16447.pdf
... Saving output csv : pos_csv/16447.csv
(6) processing : pos_reports/17993.pdf
mupdf: cannot recognize version marker
... Saving output csv : pos_csv/17993.csv
(7) processing : pos_reports/84.pdf
... Saving output csv : pos_csv/84.csv
(8) processing : pos_reports/75.pdf
... Saving output csv : pos_csv/75.csv
(9) processing : pos_reports/16423.pdf
... Saving output csv : pos_csv/16423.csv
(10) processing : pos_reports/547.pdf
... Saving output csv : pos_csv/547.csv
(11) processing : pos_reports/16375.pdf
(12) processing : pos_reports/4045.pdf
... Saving out

In [ ]:
!python -m processor.run -m pdf2csv -i neg_reports -o neg_csv

(0) processing : neg_reports/15649_2020.pdf
... Saving output csv : neg_csv/15649_2020.csv
(1) processing : neg_reports/560_2020.pdf
... Saving output csv : neg_csv/560_2020.csv
(2) processing : neg_reports/15641_2020.pdf
... Saving output csv : neg_csv/15641_2020.csv
(3) processing : neg_reports/15643_2020.pdf
... Saving output csv : neg_csv/15643_2020.csv
(4) processing : neg_reports/49_2020.pdf
... Saving output csv : neg_csv/49_2020.csv
(5) processing : neg_reports/15625_2020.pdf
... Saving output csv : neg_csv/15625_2020.csv
(6) processing : neg_reports/50_2020.pdf
... Saving output csv : neg_csv/50_2020.csv
(7) processing : neg_reports/15571_2020.pdf
... Saving output csv : neg_csv/15571_2020.csv
(8) processing : neg_reports/4005_2020.pdf
... Saving output csv : neg_csv/4005_2020.csv
(9) processing : neg_reports/557_2020.pdf
... Saving output csv : neg_csv/557_2020.csv
(10) processing : neg_reports/15579_2020.pdf
... Saving output csv : neg_csv/15579_2020.csv
(11) processing : ne

## Trim CSV files

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pos_df = pd.read_excel('Labeling.xlsx', sheet_name='Positive')
pos_df.head()

,ID,Start,End
0,8,21,28
1,9,14,15
2,9,24,31
3,11,10,17
4,12,41,52


In [ ]:
pos_df.describe()

,ID,Start,End
count,287.000000,287.000000,287.00000
mean,2929.864111,45.303136,52.97561
std,5552.824538,44.666152,46.28563
min,8.000000,1.000000,2.00000
25%,218.500000,14.000000,20.00000
50%,409.000000,30.000000,38.00000
75%,575.000000,60.500000,66.00000
max,17993.000000,288.000000,300.00000


In [ ]:
for index, row in pos_df.iterrows():
  id = row['ID']
  filename = str(id) + '.csv'
  path = 'pos_csv/' + filename
  try:
    file = pd.read_csv(path)
    file = file.loc[(file.page>=row['Start']-1) & (file.page<row['End'])].reset_index(drop=True)
    file.to_csv('preprocessed/trimmed/pos/' + str(id) + '.csv', index=False)
  except:
    print(f"The csv file of id {id} cannot be found!")

In [ ]:
neg_df = pd.read_excel('Labeling.xlsx', sheet_name='Negative')
neg_df.head()

for index, row in neg_df.iterrows():
  id = row['ID']
  filename = str(id) + '.csv'
  path = 'neg_csv/' + filename
  try:
    file = pd.read_csv(path)
    file = file.loc[(file.page>=row['Start']-1) & (file.page<row['End'])].reset_index(drop=True)
    file.to_csv('preprocessed/trimmed/neg/' + str(id) + '.csv', index=False)
  except:
    print(f"The csv file of id {id} cannot be found!")

## Preprocess report texts and concatenate

In [ ]:
import glob
import os

from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
df.content[113]

'In Quebec, we’re developing a renewable energy . ecosystem based on green hydrogen.1 And, since we  . move about 20% of the natural gas consumed in the . U.S., we’re actively exploring how much hydrogen . can be\xa0blended into our natural gas transmission and . distribution system.'

In [ ]:
a = df.copy()
a.content = a.content.str.replace(r'(\.) ([a-zA-Z])', r'\2', regex=True)

In [ ]:
a.content[113]

'In Quebec, we’re developing a renewable energy ecosystem based on green hydrogen.1 And, since we  move about 20% of the natural gas consumed in the U.S., we’re actively exploring how much hydrogen can be\xa0blended into our natural gas transmission and distribution system.'

In [ ]:
files_folder = "preprocessed/trimmed/pos"
pos_id = []
pos_texts = []

# Read all .csv files in this folder
for file in glob.glob(os.path.join(files_folder ,"*.csv")):
  df = pd.read_csv(file)
  df.content = df.content.astype(str)
  df.content = df.content.str.replace(r'(\.) ([a-zA-Z])', r'\2', regex=True)
  df.content = df.content.str.replace('#', '')
  text = ' '.join(df.content)

  text = text.lower()
  encoded_string = text.encode("ascii", "ignore")
  decode_string = encoded_string.decode()
  pos_id.append(file.split('/')[-1][:-4])
  pos_texts.append(decode_string)

In [ ]:
files_folder = "preprocessed/trimmed/neg"
neg_id = []
neg_texts = []

# Read all .csv files in this folder
for file in glob.glob(os.path.join(files_folder ,"*.csv")):
  df = pd.read_csv(file)
  df.content = df.content.astype(str)
  df.content = df.content.str.replace(r'(\.) ([a-zA-Z])', r'\2', regex=True)
  df.content = df.content.str.replace('#', '')
  text = ' '.join(df.content)

  text = text.lower()
  encoded_string = text.encode("ascii", "ignore")
  decode_string = encoded_string.decode()
  neg_id.append(file.split('/')[-1][:-9])
  neg_texts.append(decode_string)

In [ ]:
len(pos_texts), len(neg_texts)

(196, 177)

In [ ]:
pos_token = [simple_preprocess(t) for t in pos_texts]
pos_len = [len(token) for token in pos_token]

neg_token = [simple_preprocess(t) for t in neg_texts]
neg_len = [len(token) for token in neg_token]

In [ ]:
pos_len = pd.DataFrame(pos_len)
neg_len = pd.DataFrame(neg_len)

In [ ]:
pos_len[0].describe()

count      196.000000
mean      4075.913265
std       3885.487238
min         91.000000
25%       1557.750000
50%       2812.500000
75%       5052.750000
max      31778.000000
Name: 0, dtype: float64

In [ ]:
neg_len[0].describe()

count      177.000000
mean      3761.299435
std       3526.758524
min        118.000000
25%       1602.000000
50%       2782.000000
75%       4844.000000
max      22922.000000
Name: 0, dtype: float64

In [ ]:
pos_texts[0]

'19. 2019 sustainability report a sound sustainability vision is not only a powerful risk management trigger for our group but also helps us to be in-creasingly resilient to the ever-chang-ing energy scenario in which we oper-ateit shapes our approach as we seek to generate shared value in the long-term for our stakeholders. as a compass for our sustainability jour-ney, the sustainable development goals . (sdgs) help us to stay focused on our environmental, social and governance . (esg) targetsthe acceleration of cli-mate change has made the sdgs more real and urgent and the current interna-tional debate calls for a faster energy transition. we are relying on technologies, certifi-cations and our top professionals to be-come more green, all with a view to the energy transitionour goals are con-crete: minimize the carbon footprint for oil and gas processing, strengthen the circular economy and develop new products and processes for non-fossil raw materials, always encouraging the develo

## Concatenate texts

In [ ]:
pos_df = pd.DataFrame(columns=['ID', 'Text', 'Label'])
pos_df['ID'] = pos_id
pos_df['Text'] = pos_texts
pos_df['Label'] = 'Positive'

In [ ]:
neg_df = pd.DataFrame(columns=['ID', 'Text', 'Label'])
neg_df['ID'] = neg_id
neg_df['Text'] = neg_texts
neg_df['Label'] = 'Negative'

In [ ]:
pos_df

,ID,Text,Label
0,8,19. 2019 sustainability report a sound sustain...,Positive
1,11,key measures of the sustainability strategy 20...,Positive
2,9,in order to assess the environmental impact of...,Positive
3,12,105 sustainability report 2020 lenzing group s...,Positive
4,17,environmental protection classification the po...,Positive
...,...,...,...
191,595,"our values, our employees and our csr commitme...",Positive
192,3187,116 117report on non-financial data of pge pol...,Positive
193,599,january stein automation gmbh & cokgstein auto...,Positive
194,3193,53 2019 sustainability report in addition to i...,Positive


## Add company-specific info

In [ ]:
companies = pd.read_excel('Trimmed_company_list.xlsx')
companies.head()

,Unique ID,Issuer - subsidiary,Issuer industry,Country of risk,Date,Year
0,4047,Beijing Enterprises Water Capital Management H...,Utilities,China,2021-04-30,2021
1,86,SFL Corp Ltd,Industrials,Norway,2021-04-29,2021
2,16456,Dana Inc,Consumer discretionary,United States,2021-04-29,2021
3,16468,Stockholm Exergi Holding AB,Utilities,Sweden,2021-04-29,2021
4,16447,ZF Finance GmbH,Consumer discretionary,Germany,2021-04-28,2021


In [ ]:
pos_df['Company name'] = 'NA'
pos_df['Industry'] = 'NA'
pos_df['Country'] = 'NA'

for index, row in pos_df.iterrows():
  company_row = companies.loc[companies['Unique ID']==int(row['ID'])]
  if len(company_row) == 0:
    print(f"Cannot find company with ID {row['ID']}")
  name = company_row['Issuer - subsidiary'].tolist()[0]
  industry = company_row['Issuer industry'].tolist()[0]
  country = company_row['Country of risk'].tolist()[0]

  pos_df.at[index, 'Company name'] = name
  pos_df.at[index, 'Industry'] = industry
  pos_df.at[index, 'Country'] = country

In [ ]:
neg_df['Company name'] = 'NA'
neg_df['Industry'] = 'NA'
neg_df['Country'] = 'NA'

for index, row in neg_df.iterrows():
  company_row = companies.loc[companies['Unique ID']==int(row['ID'])]
  if len(company_row) == 0:
    print(f"Cannot find company with ID {row['ID']}")
  name = company_row['Issuer - subsidiary'].tolist()[0]
  industry = company_row['Issuer industry'].tolist()[0]
  country = company_row['Country of risk'].tolist()[0]

  neg_df.at[index, 'Company name'] = name
  neg_df.at[index, 'Industry'] = industry
  neg_df.at[index, 'Country'] = country

In [ ]:
pos_df.head(5)

,ID,Text,Label,Company name,Industry,Country
0,8,19. 2019 sustainability report a sound sustain...,Positive,Maire Tecnimont SpA,Industrials,Italy
1,11,key measures of the sustainability strategy 20...,Positive,Aurubis AG,Materials,Germany
2,9,in order to assess the environmental impact of...,Positive,Faber-Castell AG,Consumer discretionary,Germany
3,20,company fi nancial statements sustainablesolu...,Positive,Arcadis NV,Industrials,Netherlands
4,12,105 sustainability report 2020 lenzing group s...,Positive,Lenzing AG,Materials,Austria


In [ ]:
neg_df.head(5)

,ID,Text,Label,Company name,Industry,Country
0,9,in order to assess the environmental impact of...,Negative,Faber-Castell AG,Consumer discretionary,Germany
1,8,118creating value 7.4 environment the positive...,Negative,Maire Tecnimont SpA,Industrials,Italy
2,12,61 sustainability report 2019 | lenzing group ...,Negative,Lenzing AG,Materials,Austria
3,11,key measures of the sustainability strategy 20...,Negative,Aurubis AG,Materials,Germany
4,22,report of the management board on the operatio...,Negative,Tauron Polska Energia SA,Utilities,Poland


In [ ]:
pos_df.Industry.value_counts()

Utilities                 43
Consumer discretionary    39
Industrials               34
Materials                 27
Consumer staples          20
Energy                    11
Communications             8
Health care                7
Technology                 6
Unknown                    1
Name: Industry, dtype: int64

In [ ]:
pos_df.Country.value_counts()

France            22
United States     19
United Kingdom    14
Japan             13
Germany           13
Spain             13
Italy             10
Norway             9
China              8
Netherlands        7
South Korea        7
Canada             5
Finland            5
Singapore          4
Sweden             4
Hong Kong          4
Austria            4
Brazil             3
Belgium            3
India              3
Thailand           3
Philippines        2
Ireland            2
Malaysia           2
Taiwan             2
Mexico             2
Chile              2
Poland             2
Luxembourg         2
Russia             2
Turkey             1
Portugal           1
Argentina          1
Australia          1
New Zealand        1
Name: Country, dtype: int64

In [ ]:
pos_df.to_csv('preprocessed/Positive_reports.csv', index=False)
neg_df.to_csv('preprocessed/Negative_reports.csv', index=False)